# Content Revenue table rolling L12M

In [0]:
%sql
create or replace table dev.bi.content_rolling_rev_rec_last12M_mgtv AS
with content_agg (select rev_rec_month_start_dt as dt_month
                  ,attributed_content_item_id as content_id
                  ,sum(rev_rec_amount_usd_estimate) as rev_rec_amount
                  from prod.gold_base_restricted.estimated_combined_revenue_recognition 
                  WHERE date(rev_rec_month_start_dt) < to_date(date_trunc('month',current_date))
                  group by 1,2)
select dt_month,content_id,rev_rec_amount,
    SUM(rev_rec_amount)  OVER (PARTITION BY content_id ORDER BY dt_month ROWS BETWEEN 12 PRECEDING AND 1 PRECEDING) AS rolling_n12m_rev_rec
from content_agg

num_affected_rows,num_inserted_rows


# Revenue Features at Domain, Partner and Cluster level

In [0]:
create or replace view dev.bi.content_s12n_prdomain_rolling_rev_rec_last12M_mgtv AS
select dt_month
      ,lower(replace(content_domain,' ','-')) as content_domain
      ,count(*) as cnt
      ,sum(rev_rec_amount) as pd_sum_rolling_n12m_rev_rec
      ,avg(rev_rec_amount) as pd_avg_rolling_n12m_rev_rec
      FROM dev.bi.content_rolling_rev_rec_last12M_mgtv a
      inner join dev.bi.content_dim_kpi b on a.content_id = b.underlying_product_item_id
      where underlying_content_type = 's12n' and content_domain is not null
      group by 1,2;

create or replace view dev.bi.content_s12n_subdomain_rolling_rev_rec_last12M_mgtv AS
select dt_month
      ,lower(replace(content_sub_domain,' ','-')) as content_sub_domain
      ,sum(rev_rec_amount) as sd_sum_rolling_n12m_rev_rec
      FROM dev.bi.content_rolling_rev_rec_last12M_mgtv a
      inner join dev.bi.content_dim_kpi b on a.content_id = b.underlying_product_item_id
      where underlying_content_type = 's12n' and content_sub_domain is not null
      group by 1,2;


create or replace view dev.bi.content_s12n_partner_rolling_rev_rec_last12M_mgtv AS
select dt_month
      ,partner_id
      ,sum(rev_rec_amount) as pr_sum_rolling_n12m_rev_rec
      FROM dev.bi.content_rolling_rev_rec_last12M_mgtv a
      inner join dev.bi.content_dim_kpi b on a.content_id = b.underlying_product_item_id
      where underlying_content_type = 's12n' and partner_id is not null
      group by 1,2;

create or replace view dev.bi.content_s12n_partner_cluster_rolling_rev_rec_last12M_mgtv AS
With partner_agg (select dt_month
                  ,b.partner_id
                  ,c.cluster_rank
                  ,sum(rev_rec_amount) as pr_sum_rolling_n12m_rev_rec
                  FROM dev.bi.content_rolling_rev_rec_last12M_mgtv a
                  inner join dev.bi.content_dim_kpi b on a.content_id = b.underlying_product_item_id
                  left join dev.bi.s12n_partner_cluster_ranks_mgtv c on b.partner_id = c.partner_id
                  where underlying_content_type = 's12n' and b.partner_id is not null
                  group by 1,2,3)
select dt_month,
      cluster_rank,
      avg(pr_sum_rolling_n12m_rev_rec) as pcr_avg_rolling_n12m_rev_rec,
      sum(pr_sum_rolling_n12m_rev_rec) as pcr_sum_rolling_n12m_rev_rec
      from partner_agg
      group by 1,2;


create or replace view dev.bi.content_s12n_subdomain_pcr_rolling_rev_rec_last12M_mgtv AS
select dt_month
  ,lower(replace(content_sub_domain,' ','-')) as content_sub_domain
  ,c.cluster_rank
  ,avg(rev_rec_amount) as sd_pcr_avg_rolling_n12m_rev_rec
  ,sum(rev_rec_amount) as sd_pcr_sum_rolling_n12m_rev_rec
  FROM dev.bi.content_rolling_rev_rec_last12M_mgtv a
  inner join dev.bi.content_dim_kpi b on a.content_id = b.underlying_product_item_id
  left join dev.bi.s12n_partner_cluster_ranks_mgtv c on b.partner_id = c.partner_id
  where underlying_content_type = 's12n' and content_sub_domain is not null
  group by 1,2,3